In [34]:
#from numba import *
from numba import cuda
import numpy as np

<img src="img/cuda_indexing.png" width="480" height="480">

<img src="img/cuda_indexing_0.png" width="480" height="480">

# Hardware structure

Grid > Block > Thread

<img src="img/memory-hierarchy.png" width="240" height="240">

    - thread ID of a thread of index (x, y) is (x + y Dx)
    - the size of a block should be a multiple of 32 threads
        a) typical size is between 128 and 512 threads
    - 32 thread form a warp(?)
    -  A warp is a set of 32 threads within a thread block such that all the threads in a warp execute the same instruction
    
    - The fastest memory is registers just as in CPU. 
    - L1 cache and shared memory is second, which is also pretty limited in size.
    - Individual threads composing a warp start together at the same program address, but they have their own instruction address counter and register state and are therefore free to branch and execute independently.
    - Each warp gets scheduled by a warp scheduler for execution
    - Each warp contains threads of consecutive, increasing thread IDs with the first warp containing thread 0.
    - A warp executes one common instruction at a time, so full efficiency is realized when all 32 threads of a warp agree on their execution path.
    - If threads of a warp diverge via a data-dependent conditional branch, the warp executes each branch path taken, disabling threads that are not on that path
    - The threads of a warp that are participating in the current instruction are called the active threads, whereas threads not on the current instruction are inactive (disabled). 
    - If an atomic instruction executed by a warp reads, modifies, and writes to the same location in global memory for more than one of the threads of the warp, each read/modify/write to that location occurs and they are all serialized, but the order in which they occur is undefined. 
    - The execution context (program counters, registers, etc.) for each warp processed by a multiprocessor is maintained on-chip during the entire lifetime of the warp. Therefore, switching from one execution context to another has no cost, and at every instruction issue time, a warp scheduler selects a warp that has threads ready to execute its next instruction (the active threads of the warp) and issues the instruction to those threads. 

<b>The number of blocks and warps that can reside and be processed together on the multiprocessor for a given kernel depends on the amount of registers and shared memory used by the kernel and the amount of registers and shared memory available on the multiprocessor.</b>

The Warp Schedulers means two (4 for GTX 1050 ti?) warps can be issued at the same time. 32*4*6 = 768 threads per SM?

The <b>threads of a thread block execute concurrently on one multiprocessor</b>, and <b>multiple thread blocks can execute concurrently on one multiprocessor</b>. As thread blocks terminate, new blocks are launched on the vacated multiprocessors.

###### 1050 ti

With 128 single-precision CUDA cores per SM, GeForce GTX 1050 Ti wields a total of 768. Each SM ( of <b>6 SM </b>) also includes eight texture units (adding up to 48 across the processor), 256 KB of register file capacity, 96 KB of shared memory, and 48 KB of L1/texture cache.

<b>L1, L2, L3 caches</b>

Кэш память первого уровня L1 — самая маленькая по объему (всего несколько десятков килобайт), но самая быстрая по скорости и наиболее важная. Она содержит данные наиболее часто используемые процессором и работает без задержек. Обычно количество микросхем памяти уровня L1 равно количеству ядер процессора, при этом каждое ядро получает доступ только к своей микросхеме L1.

Кэш память уровня L2 по скорости уступает памяти L1, но выигрывает в объеме, который измеряется уже в нескольких сотнях килобайт. Она предназначена для временного хранения важной информации, вероятность обращения к которой ниже, чем у информации хранящейся в кэше L1.

Третий уровень кэш памяти L3 — имеет самый большой объем из трех уровней (может достигать десятков мегабайт), но и обладает самой медленной скоростью, которая всё же значительно выше скорости оперативной памяти. Кэш память L3 служит общей для всех ядер процессора. Уровень памяти L3 предназначен для временного хранения тех важных данных, вероятность обращения к которым чуть ниже, чем у информации которая хранится в первых двух уровнях L1, L2. Она также обеспечивает взаимодействие ядер процессора между собой. Некоторые модели процессоров выполнены с двумя уровнями кэш памяти, в которых L2 совмещает все функции L2 и L3.


###### Memory

    - global device memory
    - shared memory
    - local memory


On the software side, the block size determines how many threads share a given area of shared memory.

The block size you choose depends on a range of factors, including:

    The size of the data array
    The size of the shared mempory per block (e.g. 64KB)
    The maximum number of threads per block supported by the hardware (e.g. 512 or 1024)
    The maximum number of threads per multiprocessor (MP) (e.g. 2048)
    The maximum number of blocks per MP (e.g. 32)
    The number of threads that can be executed concurrently (a “warp” i.e. 32)

NVIDIA recommends that programmers focus on following those recommendations to achieve the best performance:

    Find ways to parallelize sequential code
    Minimize data transfers between the host and the device
    Adjust kernel launch configuration to maximize device utilization
    Ensure global memory accesses are coalesced
    Minimize redundant accesses to global memory whenever possible
    Avoid different execution paths within the same warp


###### <b>Kernel</b>

A kernel function is a GPU function that is meant to be called from CPU code. It has two fundamental characteristics:

    kernels cannot explicitly return a value; all result data must be written to an array passed to the function (if computing a scalar, you will probably pass a one-element array);
    kernels explicitly declare their thread hierarchy when called: i.e. the number of thread blocks and the number of threads per block (note that while a kernel is compiled once, it can be called multiple times with different block sizes or grid sizes).


Number of registers per thread that kernel used?

###### Code

In [12]:
cuda.is_available()
#cuda.get_memory_info()

True

In [16]:
import numpy as np

@cuda.jit
def max_example(result, values):
#   Find the maximum value in values and store in result[0]
    tid = cuda.threadIdx.x
    bid = cuda.blockIdx.x
    bdim = cuda.blockDim.x
    i = (bid * bdim) + tid
    cuda.atomic.max(result, 0, values[i])


arr = np.random.rand(16384)
result = np.zeros(1, dtype=np.float64)

# Max # of resident threads per SM is 2048 for 1050 ti !!!
# Max # of threads per block 1024 !!!
max_example[4,1024](result, arr) 
print(result[0]) # Found using cuda.atomic.max
print(max(arr))  # Print max(arr) for comparision (should be equal!)

0.9994201512378423
0.9998168949194969


https://www.youtube.com/watch?v=7iE0sHiGGxs Occupancy limit

###### Example of parameters calculation

How many thread blocks per SM can we run?

1 dim grid

In [19]:
import math

# CUDA kernel
@cuda.jit
def my_kernel(io_array):
    
    #Return the absolute position of the current thread in the entire grid of blocks.
    pos = cuda.grid(1) 
    if pos < io_array.size:
        io_array[pos] *= 2 # do the computation

# Host code   
data = np.ones(256)
threadsperblock = 256
blockspergrid = math.ceil(data.shape[0] / threadsperblock)
my_kernel[blockspergrid, threadsperblock](data)
print(data)

[2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]


2 dim grid

In [20]:
from __future__ import division
from numba import cuda
import numpy
import math

@cuda.jit
def my_kernel_2D(io_array):
    x, y = cuda.grid(2)
    if x < io_array.shape[0] and y < io_array.shape[1]:
        io_array[x,y] *= 2 # do the computation

data= np.ones((16, 16))
threadsperblock = (16, 16)
blockspergrid_x = math.ceil(data.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(data.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)
my_kernel_2D[blockspergrid, threadsperblock](data)
print(data)

[[2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]]


In [23]:
device_array = cuda.device_array(2)
type(device_array)

numba.cuda.cudadrv.devicearray.DeviceNDArray

In [25]:
device_array = cuda.to_device([1,2,3])
type(device_array)

numba.cuda.cudadrv.devicearray.DeviceNDArray

<b>Example Matrix multipliction</b>

In [26]:
from __future__ import division
from numba import cuda
import numpy
import math

# CUDA kernel
@cuda.jit
def matmul(A, B, C):
    """Perform matrix multiplication of C = A * B
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp
        
# Host code

# Initialize the data arrays
A = numpy.full((24, 12), 3, numpy.float) # matrix containing all 3's
B = numpy.full((12, 22), 4, numpy.float) # matrix containing all 4's

# Copy the arrays to the device
A_global_mem = cuda.to_device(A)
B_global_mem = cuda.to_device(B)

# Allocate memory on the device for the result
C_global_mem = cuda.device_array((24, 22))

# Configure the blocks
threadsperblock = (16, 16)
blockspergrid_x = int(math.ceil(A.shape[0] / threadsperblock[0]))
blockspergrid_y = int(math.ceil(B.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

# Start the kernel 
matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)

# Copy the result back to the host
C = C_global_mem.copy_to_host()

print(C)

[[144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144.
  144. 144. 144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144.
  144. 144. 144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144.
  144. 144. 144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144.
  144. 144. 144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144.
  144. 144. 144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144.
  144. 144. 144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144.
  144. 144. 144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144.
  144. 144. 144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144. 144.
  144. 1

! A problem with this code is that each thread is <b>reading from the global memory</b> containing the copies of A and B.
In fact, the A global memory is read B.shape[1] times and the B global memory is read A.shape[0] times. Since global memory is fairly slow, this results in an inefficient use of the GPU.

###### Shared memory and thread synchronization

In [27]:
# cuda.shared.array
# cuda.syncthreads() # This function will synchronize all threads in the same thread block.

The memory is allocated once for the duration of the kernel, unlike traditional dynamic memory management.

Because the shared memory is a limited resource, it is often necessary to preload a small block at a time from the input arrays. All the threads then need to wait until everyone has finished preloading before doing the computation on the shared memory.

Synchronization is then required again after the computation to ensure all threads have finished with the data in shared memory before overwriting it in the next loop iteration.

###### Advanced matrix multiplication

In [29]:
from numba import float32

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

@cuda.jit
def fast_matmul(A, B, C):
    """
    Perform matrix multiplication of C = A * B
    Each thread computes one element of the result matrix C
    """

    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)
    
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    
    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(int(A.shape[1] / TPB)):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

# The data array
A = numpy.full((TPB*2, TPB*3), 3, numpy.float) # [32 x 48] matrix containing all 3's
B = numpy.full((TPB*3, TPB*1), 4, numpy.float) # [48 x 16] matrix containing all 4's

A_global_mem = cuda.to_device(A)
B_global_mem = cuda.to_device(B)
C_global_mem = cuda.device_array((TPB*2, TPB*1)) # [32 x 16] matrix result

# Configure the blocks
threadsperblock = (TPB, TPB)
blockspergrid_x = int(math.ceil(A.shape[0] / threadsperblock[1]))
blockspergrid_y = int(math.ceil(B.shape[1] / threadsperblock[0]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

# Start the kernel 
fast_matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)
res = C_global_mem.copy_to_host()

print(res)

[[576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576

<b>Kernel doesn't require specify type of itself and the arguments</b>

In [39]:
np.size?